In [1]:
import logging
import os
from pathlib import Path
import shutil

from cm1.run import PBS, CM1Run
from cm1.utils import TMPDIR
import f90nml

In [2]:
# Path to the CM1 repository
CM1 = Path(os.getenv("SCRATCH")) / "cm1"  # Assumes $SCRATCH is already set.

testcase = "les_ib_windtunnel"
run_dir = CM1 / f"run_{testcase}"
os.makedirs(run_dir, exist_ok=True)

# Copy all files from testcase configuration directory to run directory
shutil.copytree(CM1 / "run/config_files" / testcase, run_dir, dirs_exist_ok=True)

# Initialize the controller with the path to the namelist file
namelist_path = run_dir / "namelist.input"
namelist = f90nml.read(namelist_path)

# Modify values in the namelist
namelist["param9"]["output_format"] = 2
namelist["param9"]["output_filetype"] = 1
namelist["param2"]["isnd"] = 7

# Write updated namelist in run directory.
namelist.write(namelist_path, force=True)
logging.info(f"Updated namelist saved to {namelist_path}")

# Link LW and SW radiation data and model executable to run directory
for file in ["RRTMG_LW_DATA", "RRTMG_SW_DATA", "cm1.exe"]:
    symlink = run_dir / file
    if os.path.exists(symlink):
        continue
    symlink.symlink_to(CM1 / f"run/{file}")

pbs_config = PBS(
    name=testcase,
    account="NMMM0021",
    walltime="02:00:00",
    nodes=2,
    run_dir=run_dir,
)
cm1_run = CM1Run(namelist=namelist, pbs_config=pbs_config)
cm1_run.submit_job()

7666309.desched1


In [12]:
namelist["param0"]

Namelist([('nx', 1144),
          ('ny', 1144),
          ('nz', 106),
          ('ppnode', 128),
          ('timeformat', 3),
          ('timestats', 1),
          ('terrain_flag', False),
          ('procfiles', False)])

In [13]:
cm1_run